In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder


from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from xgboost import XGBClassifier

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/test.csv')

In [3]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [4]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [5]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [6]:
RF = RandomForestClassifier(n_estimators=100, random_state=37).fit(train_x, train_y)

ADA = AdaBoostClassifier(n_estimators=100, random_state=37).fit(train_x, train_y)

XGB = XGBClassifier(n_estimators=100, random_state=37).fit(train_x, train_y)

In [7]:
#최종모델
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()

In [8]:
RF_preds = RF.predict(test_x)
ADA_preds = ADA.predict(test_x)
XGB_preds = XGB.predict(test_x)

In [9]:
new_data = np.array([RF_preds, ADA_preds, XGB_preds])
new_data.shape

(3, 310)

In [10]:
new_data = np.transpose(new_data)
new_data.shape

(310, 3)

In [16]:
LGBM = LGBMClassifier(n_estimators=100, random_state=37).fit(train_x, train_y)

In [17]:
eclf = VotingClassifier(estimators=[('rf', RF), ('ada', ADA), ('lgbm', LGBM)],voting='soft')

In [19]:
eclf = eclf.fit(train_x, train_y)

In [20]:
preds = eclf.predict(test_x)

In [21]:
submit = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/sample_submission.csv')

submit['Y_Class'] = preds

submit.to_csv('/content/drive/MyDrive/LG Aimers 2기/제출/baseline_submission5.csv', index=False)